<a href="https://colab.research.google.com/github/colincockburn/CISC_473_project/blob/main/image_restoration_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell at start of each session - Change BRANCH name to your working branch.
# if pushing changes to this cell make sure to add them to both notebooks version of it.

import yaml, os
from google.colab import drive

# ==== Team Bootstrap (Colab) ====
# 1) Mount Drive (data only)
drive.mount('/content/drive')

# 2) define consts
REPO_URL  = "https://github.com/colincockburn/CISC_473_project.git"
REPO_DIR  = "/content/drive/MyDrive/projects/CISC_473_project/"
DATA_ROOT = "/content/drive/MyDrive/projects/CISC_473_project_data"

# 3) Clone repo if missing
import os, subprocess, sys
if not os.path.isdir(REPO_DIR):
    os.makedirs(os.path.dirname(REPO_DIR), exist_ok=True)
    print("Cloning repo to", REPO_DIR)
    !git clone "$REPO_URL" "$REPO_DIR"

# 4) Git identity (first time only; uncomment and change to your info)'
# dont push changes

%cd $REPO_DIR
!git config user.email "colincockburn64@gmail.com"
!git config user.name "colincockburn"

# 5) Sync with origin/main and create/update your feature branch
BRANCH = "cc-training"  # e.g., feature/cc-dataloader
if BRANCH:
  !git fetch origin
  !git checkout main
  !git pull origin main
  !git checkout -B "$BRANCH"
else:
  raise Exception("Please update BRANCH var to your desired working branch")

# 6) Sanity checks
import os
assert os.path.isdir(DATA_ROOT), f"Missing data at: {DATA_ROOT}"
print("OK: repo at", REPO_DIR)
print("OK: data at", DATA_ROOT)
!git status


KeyboardInterrupt: 

In [ ]:
# get configs
with open(f"{REPO_DIR}/configs/default.yaml", "r") as f:
    cfg = yaml.safe_load(f)

In [ ]:
# run training
import sys
sys.path.append("/content/drive/MyDrive/projects/CISC_473_project/src")  # adjust if needed
from train import main

!mkdir -p ../checkpoints
data_root = cfg["paths"]["data_root"]
save_dir = cfg["paths"]["save_dir"]
# Simulate command-line arguments directly inside the notebook
sys.argv = [
    "train.py",
    "--data_root", data_root,
    "--save_dir", save_dir,
    "--epochs", "10",
    "--batch_size", "16",
    "--patch_size", "128",
    "--lr", "1e-3"
]

main()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: '

In [ ]:
# monitor metrics

In [ ]:
# eval model

In [ ]:
# disply results